## Export data to SpaceTX format

First create the subclasses of `FetchedTile` and `TileFetcher` required and then export the data into SpaceTX

In [1]:
%load_ext autoreload
%autoreload 2

import functools
import os
from typing import Mapping, Tuple, Union

import click
import numpy as np
from skimage.io import imread
from slicedimage import ImageFormat

from starfish import Codebook
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue, Features

import pdb

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return imread(file_path)

class StarMapTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            coordinates: Mapping[Union[str, Coordinates], CoordinateValue]
    ) -> None:
        self.file_path = file_path
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
        print(np.shape(self.tile_data()))
#         return {Axes.Y: 7962, Axes.X: 3356}
        return {Axes.Y: 4000, Axes.X: 800}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    @staticmethod
    def crop(img) -> np.ndarray:
        crp = img[2000:6000, 1000:1800]
        return crp
    
    def tile_data(self) -> np.ndarray:
        return self.crop(imread(self.file_path))
    
    def __str__(self) -> str:
        return self.file_path
    
class StarMapTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
#         self.num_z = 11
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round{round_label+1}_2x10_1_FusionStitcher_C{ch_label+1}_Z{zplane_padded}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
class StarMapDAPITileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round{round_label+1}_2x10_1_FusionStitcher_C{ch_label}_Z{zplane_padded}.tif"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
def format_data(input_dir, output_dir):
    
    primary_image_dimensions: Mapping[Axes, int] = {
        Axes.ROUND: 1,
        Axes.CH: 4,
        Axes.ZPLANE: 11,
    }
    
    aux_image_dimensions: Mapping[str, Mapping[Union[str, Axes], int]] = {
        "nuclei": {
            Axes.ROUND: 1,
            Axes.CH: 1,
            Axes.ZPLANE: 11
        }
    }
    
#     pdb.set_trace()
    write_experiment_json(
        path=output_dir,
        fov_count=1,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        aux_name_to_dimensions=aux_image_dimensions,
        primary_tile_fetcher=StarMapTileFetcher(input_dir),
        aux_tile_fetcher={"nuclei": StarMapDAPITileFetcher(input_dir)},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    
    codebook = [
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "GFP"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 1, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "RFP"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 2, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "Cy5"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 3, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "iRFP"
        }
    ]
    Codebook.from_code_array(codebook).to_json("/home/nomi/Desktop/starfish/experiment/codebook.json")

In [13]:
%load_ext autoreload
%autoreload 2

import functools
import os
from typing import Mapping, Tuple, Union

import click
import numpy as np
from skimage.io import imread
from slicedimage import ImageFormat

from starfish import Codebook
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue, Features

import pdb

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return imread(file_path)

class StarMapTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            coordinates: Mapping[Union[str, Coordinates], CoordinateValue]
    ) -> None:
        self.file_path = file_path
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
        print(np.shape(self.tile_data()))
#         return {Axes.Y: 7962, Axes.X: 3356}
        return {Axes.Y: 4000, Axes.X: 800}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    @staticmethod
    def crop(img) -> np.ndarray:
        crp = img[2000:6000, 1000:1800]
        return crp
    
    def tile_data(self) -> np.ndarray:
        return self.crop(imread(self.file_path))
    
    def __str__(self) -> str:
        return self.file_path
    
class StarMapTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
#         self.num_z = 11
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round{round_label+1}_2x10_1_FusionStitcher_C2_Z{zplane_padded}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
class StarMapDAPITileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round{round_label+1}_2x10_1_FusionStitcher_C{ch_label}_Z{zplane_padded}.tif"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
def format_data(input_dir, output_dir):
    
    primary_image_dimensions: Mapping[Axes, int] = {
        Axes.ROUND: 1,
        Axes.CH: 1,
        Axes.ZPLANE: 11,
    }
    
    aux_image_dimensions: Mapping[str, Mapping[Union[str, Axes], int]] = {
        "nuclei": {
            Axes.ROUND: 1,
            Axes.CH: 1,
            Axes.ZPLANE: 11
        }
    }
    
#     pdb.set_trace()
    write_experiment_json(
        path=output_dir,
        fov_count=1,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        aux_name_to_dimensions=aux_image_dimensions,
        primary_tile_fetcher=StarMapTileFetcher(input_dir),
        aux_tile_fetcher={"nuclei": StarMapDAPITileFetcher(input_dir)},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    codebook = [
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "RFP"
        }
    ]
    Codebook.from_code_array(codebook).to_json("/home/nomi/Desktop/starfish/experiment/codebook.json")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
format_data("/home/nomi/Desktop/starfish/raw_data/2019-06-29_Justus_section3_round1_2x10_1",
           "/home/nomi/Desktop/starfish/experiment")

(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)


/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718410> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba7186d0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718780> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718af0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718200> is a low contrast image
  warn('%s is a

(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)
(4000, 800)


/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718ca8> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718d58> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718e08> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718308> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7fe4ba718f68> is a low contrast image
  warn('%s is a

## Load the experiment and visualize the codebook
Possibly necessary to copy the codebook information from `codebook_backup.json`.

In [15]:
%load_ext autoreload
%autoreload 2
from starfish import Experiment
from six.moves import urllib
from slicedimage.backends import CachingBackend, DiskBackend, HttpBackend, SIZE_LIMIT

baseurl="/home/nomi/Desktop/starfish/experiment/experiment.json"
experiment = Experiment.from_json(baseurl)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
experiment.codebook

<xarray.Codebook (target: 1, c: 1, r: 1)>
array([[[1]]], dtype=uint8)
Coordinates:
  * target   (target) object 'RFP'
  * c        (c) int64 0
  * r        (r) int64 0

## Visualize the FOV

In [17]:
fov = experiment["fov_000"]

In [18]:
image = fov.get_image("primary")

100%|██████████| 11/11 [00:00<00:00, 28.56it/s]


In [5]:
import starfish
%gui qt
ipython = get_ipython()
ipython.magic("gui qt5")
starfish.display(image)

## Project onto the Z axis and clip
The slider in the viewer should now only have as many options as there are channels. Also clipping the image to remove background noise.

In [19]:
import starfish
from starfish.types import Axes
clipper = starfish.image.Filter.Clip(p_min=97)
z_proj: starfish.ImageStack = clipper.run(image.max_proj(Axes.ZPLANE))

100%|██████████| 1/1 [00:00<00:00, 131.05it/s]


In [7]:
starfish.display(z_proj)

Find spots
----------
Finally, a local blob detector that finds spots in each (z, y, x) volume
separately is applied. The user selects an "anchor round" and spots found in
all channels of that round are used to seed a local search across other rounds
and channels. The closest spot is selected, and any spots outside the search
radius (here 10 pixels) is discarded.

The Spot finder returns an IntensityTable containing all spots from round
zero. Note that many of the spots do _not_ identify spots in other rounds and
channels and will therefore fail decoding. Because of the stringency built
into the STARmap codebook, it is OK to be relatively permissive with the spot
finding parameters for this assay.

In [20]:
import starfish
import numpy as np
# lsbd = starfish.spots.DetectSpots.BlobDetector(
#     min_sigma=.5,
#     max_sigma=8,
#     num_sigma=50,
#     threshold=np.percentile(np.ravel(z_proj.xarray.values), 95),
# #     exclude_border=2,
# #     anchor_round=0,
# #     search_radius=10,
# )
# tlmpf = starfish.spots.DetectSpots.TrackpyLocalMaxPeakFinder(
#     spot_diameter=1,  # must be odd integer
#     min_mass=0.02,
#     max_size=4,  # this is max radius
#     separation=1,
#     noise_size=0.65,  # this is not used because preprocess is False
#     preprocess=False,
#     percentile=10,  # this is irrelevant when min_mass, spot_diameter, and max_size are set properly
#     verbose=True,
#     is_volume=True,
# )
lmpf = starfish.spots.DetectSpots.LocalMaxPeakFinder(
min_distance=1,
stringency=0,
min_obj_area=1,
max_obj_area=20,
verbose=True,
is_volume=True)

#intensities = lsbd.run(z_proj, n_processes=22)
# intensities = tlmpf.run(clipper.run(image), n_processes=22)
intensities = lmpf.run(z_proj, n_processes=22)

  0%|          | 0/100 [00:00<?, ?it/s]

Determining optimal threshold ...


 94%|█████████▍| 94/100 [00:16<00:01,  5.96it/s]

Stopping early at threshold=0.005619327476567993. Number of spots fell below: 3
computing final spots ...


Decode spots
------------
Next, spots are decoded.

In [21]:
decoded = experiment.codebook.decode_per_round_max(intensities.fillna(0))
decode_mask = decoded['target'] != 'nan'

In [38]:
import sklearn
print(sorted(sklearn.neighbors.VALID_METRICS['ball_tree']))
decoded = experiment.codebook.decode_metric(intensities.fillna(0), 1, 0, 2, 'l2')
decode_mask = decoded['target'] != 'nan'

['braycurtis', 'canberra', 'chebyshev', 'cityblock', 'dice', 'euclidean', 'hamming', 'haversine', 'infinity', 'jaccard', 'kulsinski', 'l1', 'l2', 'mahalanobis', 'manhattan', 'matching', 'minkowski', 'p', 'pyfunc', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'wminkowski']


In [22]:
import pandas as pd
from starfish.types import Features
from starfish.core.types import DecodedSpots

df = pd.DataFrame(dict(decoded['features'].coords))
pixel_coordinates = pd.Index(['x', 'y', 'z'])
ds = DecodedSpots(df)

In [23]:
ds.data

,radius,x,y,z,features,xc,yc,zc,target,distance,passes_thresholds
0,1,252,3999,0,0,0.000032,1.000000e-04,0.00005,RFP,0.0,True
1,1,614,3978,0,1,0.000077,9.947487e-05,0.00005,RFP,0.0,True
2,1,612,3978,0,2,0.000077,9.947487e-05,0.00005,RFP,0.0,True
3,1,610,3976,0,3,0.000076,9.942486e-05,0.00005,RFP,0.0,True
4,1,609,3975,0,4,0.000076,9.939985e-05,0.00005,RFP,0.0,True
5,1,611,3974,0,5,0.000076,9.937484e-05,0.00005,RFP,0.0,True
6,1,610,3972,0,6,0.000076,9.932483e-05,0.00005,RFP,0.0,True
7,1,608,3972,0,7,0.000076,9.932483e-05,0.00005,RFP,0.0,True
8,1,600,3971,0,8,0.000075,9.929982e-05,0.00005,RFP,0.0,True
9,1,595,3971,0,9,0.000074,9.929982e-05,0.00005,RFP,0.0,True


In [24]:
%gui qt
viewer = starfish.display(
    z_proj, decoded[decode_mask], radius_multiplier=10, mask_intensities=0.00
)

In [68]:
ds.data.loc[1]

radius                         2
x                             11
x_max                         13
x_min                         10
y                           3987
y_max                       3989
y_min                       3986
z                              0
z_max                          1
z_min                          0
features                       1
xc                   1.37672e-06
yc                   9.96999e-05
zc                         5e-05
target                       GFP
distance                       0
passes_thresholds           True
Name: 1, dtype: object

In [28]:
for i,j in ds.data.iterrows():
    if j['target'] == 'RFP':
        print(j)

radius                         1
x                            612
x_max                        613
x_min                        612
y                           3960
y_max                       3961
y_min                       3960
z                              0
z_max                          1
z_min                          0
features                    4207
xc                   7.65957e-05
yc                   9.90248e-05
zc                         5e-05
target                       RFP
distance                       0
passes_thresholds           True
Name: 4207, dtype: object
radius                         1
x                            601
x_max                        602
x_min                        601
y                           3956
y_max                       3957
y_min                       3956
z                              0
z_max                          1
z_min                          0
features                    4208
xc                    7.5219e-05
yc               

In [100]:
image.axis_labels(Axes.CH)

[0, 1, 2, 3]

In [101]:
data, axes = image.get_slice({Axes.CH: 1})

In [47]:
z_proj.to_multipage_tiff('/home/nomi/Desktop/sec3_ch2_clipped.tiff')